<a href="https://colab.research.google.com/github/rastringer/promptcraft_notebooks/blob/main/llama2_celeb_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2 on Vertex AI for content evaluation

In this notebook, we will deploy a Llama2-13b model on Vertex AI to summarize and check for certain content from web links.

<p align="center">
  <img src="https://github.com/rastringer/promptcraft_notebooks/blob/main/images/llama.png?raw=true" width="600"/>
</p>

Install GCP libraries

In [ ]:
#! pip install --upgrade google-cloud-aiplatform

Authenticate (colab)

In [ ]:
from google.colab import auth
auth.authenticate_user()

Set project id and location / region

In [ ]:
from google.cloud import aiplatform

PROJECT_ID = "<..>"
LOCATION = "<..>"

### Deploy model

The next step is easily completed within the [GCP console](https://console.google.com).

From the main menu, go to Vertex AI, then the Model Garden page.

Search for 'Llama2'. On its model card, click View details.

Click Deploy.

You're asked to save a copy of the model to Model Registry.

In Model name, enter a name for the model.

Click Save.

The Deploy to endpoint pane appears.

Define your endpoint as follows:

Endpoint name: Enter a name for the endpoint.
Region: Select a region to create the endpoint in.
Access: To make the endpoint accessible through a REST API, select radio_button_checked Standard. To create a private connect to the endpoint, select radio_button_checked Private.
Click Continue.

Follow the instructions in the Google Cloud console and configure your model settings.

Click Continue.

Optional: Click the Enable model monitoring for this endpoint toggle to enable model monitoring.

Click Deploy. This should take 10-15 minutes.

Now we can use the SDK to find the model and to send requests.

In [ ]:
vertex_client = aiplatform.gapic.EndpointServiceClient(
    client_options={"api_endpoint": "us-central1-aiplatform.googleapis.com"}
)

In [ ]:
request = aiplatform.gapic.ListEndpointsRequest(parent=f"projects/{PROJECT_ID}/locations/{LOCATION}")
response = vertex_client.list_endpoints(request)
endpoints = response.endpoints

Check the list of endpoints for the endpoint ID. This is the number at the end of the [name...] entry.

For example, the first line should look like this:

`[name: "projects/PROJECT_ID/locations/LOCATION/endpoints/49167192343274772480"`

and we would copy the `49167192343274772480` number as the endpoint_name referenced below.

In [143]:
endpoints

In [ ]:
# Get the endpoint ID shown above

endpoint_name = "" # @param {type:"string"}

aip_endpoint_name = (
    f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{endpoint_name}"
)
llama_endpoint = aiplatform.Endpoint(aip_endpoint_name)

In [ ]:
instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = llama_endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
What is a car?
Output:
A car (or automobile) is a wheeled motor vehicle that has seats for multiple people, typically used for transportation on roads and highways. Cars are equipped with an engine, transmission, and brakes, and are


### Scraping Google News

This task may be best left to the professionals, since Google News frequently updates its layouts and HTML / CSS tags, which will often break web scraping code.

[Scraping Bee](https://https://app.scrapingbee.com/) offers a free trial, then fees of between $49-$249 a month depending on your individual use or organization.

Add your API key below.

In [ ]:
import requests

def send_request():
    response = requests.get(
        url='https://app.scrapingbee.com/api/v1/store/google',
        params={
            'api_key': '<..>',

            'search': 'Taylor Swift',
            'country_code': 'gb',
            'language': 'en',
            'nb_results': '10',
            'search_type': 'news'
        },

    )

    return response.content

json_response = send_request()

This function outputs the top 10 news items for a particular celebrity / influencer / public figure etc as JSON.

In [ ]:
json_response

b'{"meta_data": {"url": "https://www.google.com/search?q=Taylor+Swift&gl=gb&hl=en&tbm=nws&num=10", "number_of_results": 3930000, "location": null, "number_of_organic_results": 0, "number_of_ads": 0, "number_of_page": 3930000}, "organic_results": [], "local_results": [], "top_ads": [], "bottom_ads": [], "related_queries": [], "questions": [], "top_stories": [], "news_results": [{"title": "Taylor Swift adds two more Wembley dates to Eras tour, equalling Take That record", "link": "https://www.bbc.co.uk/news/entertainment-arts-67413573", "source": "BBC", "date": "2023-11-14T13:55:04.623Z", "position": 1, "domain": "www.bbc.co.uk", "snippet": null}, {"title": "Taylor Swift announces two extra London dates for Eras Tour", "link": "https://news.sky.com/story/taylor-swift-announces-two-extra-london-dates-for-eras-tour-13008033", "source": "Sky News", "date": "2023-11-14T12:55:04.626Z", "position": 2, "domain": "news.sky.com", "snippet": null}, {"title": "Stranded Swiftie drama at Taylor Swift

### Parsing the response

One of the strengths of LLMs is their ability to parse links without the user supplying the text of the story. Let's simply extract the links from the JSON as a list.

In [127]:
import json

def get_links(json_input):
  data = json.loads(json_input)
  links = []
  for story in data['news_results']:
    links.append(story['link'])

  return links

get_links(json_response)

['https://www.bbc.co.uk/news/entertainment-arts-67413573',
 'https://news.sky.com/story/taylor-swift-announces-two-extra-london-dates-for-eras-tour-13008033',
 'https://www.independent.co.uk/news/world/americas/taylor-swift-argentina-eras-concert-b2446085.html',
 'https://www.buzzfeednews.com/article/stephaniesoteriou/taylor-swift-travis-kelce-public-relationship-normal',
 'https://www.dailymail.co.uk/sport/nfl/article-12740341/Travis-Kelce-McDonalds-Argentina-private-jet-Taylor-Swift.html',
 'https://www.dailymail.co.uk/sport/nfl/article-12752477/Travis-Kelce-Taylor-Swifts-Eras-Karma-Argentina-Chiefs.html',
 'https://www.forbes.com/sites/hughmcintyre/2023/11/14/taylor-swift-is-inching-closer-to-a-billboard-chart-record-no-one-wants/',
 'https://www.independent.co.uk/arts-entertainment/music/news/taylor-swift-crowd-throwing-things-argentina-b2446524.html',
 'https://www.foxnews.com/entertainment/taylor-swift-jokes-witchcraft-never-beating-sorcery-allegations-crazy-concert-moment',
 'ht

Now we can use that list in prompts to the model.

In [ ]:
instances = [
    {
        "prompt": f"Summarize the story in the following link: {links[1]}",
        "max_tokens": 1000,
        "temperature": 0.1,
        "top_p": 0.8,
        "top_k": 40,
    },
]
response = llama_endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
Summarize the story in the following link: https://news.sky.com/story/taylor-swift-announces-two-extra-london-dates-for-eras-tour-13008033
Output:
Taylor Swift has announced two extra dates for her Eras Tour in London. The pop star, who is currently on her world tour, has added two more shows at the O2 Arena in London on June 28 and 29. The additional dates come after high demand for tickets to her previous shows in the city. Tickets for the new dates will go on sale on April 24.


In [ ]:
instances = [
    {
        "prompt": f"Is there anything controversial in the following link: {links[1]}",
        "max_tokens": 800,
        "temperature": 0.2,
        "top_p": 0.8,
        "top_k": 40,
    },
]
response = llama_endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
Is there anything controversial in the following link: https://news.sky.com/story/taylor-swift-announces-two-extra-london-dates-for-eras-tour-13008033
Output:
I don't see anything controversial in the link you provided. It appears to be a straightforward news article about Taylor Swift adding two extra dates to her upcoming tour in London. The article includes information about the dates, the venues, and how fans can purchase tickets. There is no mention of any controversial topics or issues.


In [ ]:
instances = [
    {
        "prompt": f"""Tell me if there anything seriously controversial, the sort of controversy \
        that would be very negative publicity, in the following links: {links}""",
        "max_tokens": 800,
        "temperature": 0.2,
        "top_p": 0.8,
        "top_k": 40,
    },
]
response = llama_endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
Tell me if there anything seriously controversial, the sort of controversy \ 
        that would be very negative publicity, in the following links: ['https://www.bbc.co.uk/news/entertainment-arts-67413573', 'https://news.sky.com/story/taylor-swift-announces-two-extra-london-dates-for-eras-tour-13008033', 'https://www.independent.co.uk/news/world/americas/taylor-swift-argentina-eras-concert-b2446085.html', 'https://www.buzzfeednews.com/article/stephaniesoteriou/taylor-swift-travis-kelce-public-relationship-normal', 'https://www.dailymail.co.uk/sport/nfl/article-12740341/Travis-Kelce-McDonalds-Argentina-private-jet-Taylor-Swift.html', 'https://www.dailymail.co.uk/sport/nfl/article-12752477/Travis-Kelce-Taylor-Swifts-Eras-Karma-Argentina-Chiefs.html', 'https://www.forbes.com/sites/hughmcintyre/2023/11/14/taylor-swift-is-inching-closer-to-a-billboard-chart-record-no-one-wants/', 'https://www.independent.co.uk/arts-entertainment/music/news/taylor-swift-crowd-throwing-things-argenti

### Let's try it on a blog

With thanks to Gwyneth Paltrow...

In [138]:
instances = [
    {
        "prompt": f"""Tell me if there anything seriously controversial \
        in the articles in this blog: https://www.artspace.com \
        answer a Yes or No and provide a one line summary""",
        "max_tokens": 800,
        "temperature": 0.2,
        "top_p": 0.8,
        "top_k": 40,
    },
]
response = llama_endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
Tell me if there anything seriously controversial         in the articles in this blog: https://www.artspace.com         answer a Yes or No and provide a one line summary
Output:
 of the most controversial article.

I have gone through the articles on the blog you provided, and I can confidently say that there is nothing seriously controversial in any of them. The articles are all focused on art and design, and they do not touch on any sensitive or controversial topics.

However, if I had to choose one article that might be considered the most controversial, it would be "The Art of the Deal: How to Sell Your Art Online" because it discusses the business side of art and the importance of pricing and marketing one's work effectively. Some artists might find this topic to be controversial because they believe that the focus should be solely on the art itself, rather than the business aspects.

So, to summarize, there is nothing seriously controversial in the articles on this blog,

In [139]:
instances = [
    {
        "prompt": "Could you summarize the author of the blog's politics?",
        "max_tokens": 800,
        "temperature": 0.2,
        "top_p": 0.8,
        "top_k": 40,
    },
]
response = llama_endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Prompt:
Could you summarize the author of the blog's politics?
Output:
The author of the blog is a conservative and expresses views that are generally associated with the political right. However, the author also frequently criticizes both major political parties and expresses frustration with the political process. The author's views on specific issues, such as immigration and national security, are not always clear-cut or consistent with traditional conservative positions.

The author's political beliefs can be summarized as:

1. Critical of both major political parties and the political process.
2. Generally conservative, but with a willingness to challenge traditional conservative positions on specific issues.
3. Focused on issues related to national security, immigration, and the rule of law.
4. Supportive of limited government and individual freedom, but with a recognition of the need for government intervention in certain areas.
5. Skeptical of the media and the political establ